In [1]:
import torch
device = torch.device('cuda')
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
from PIL import Image
import numpy as np
#import segmentation_models_pytorch as smp
from torchvision.transforms import Normalize
from math import pi, sin,cos


In [2]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

#normalize = torchvision.transforms.Normalize(mean, std)
#normalize_fuc = Normalize(mean, std) 
def preprocess(image):
    image = Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [3]:
# load best saved checkpoint
model = torch.load('./best_model160_5.pth')
model = model.cuda().eval().half()

In [4]:
import traitlets
import ipywidgets.widgets as widgets
import ipywidgets
#from fastseg.image import colorize, blend
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
#import cv2
import time



mtx = np.array([[185.40929109,   0.        , 219.49061385],
       [  0.        , 243.56364667, 211.93760043],
       [  0.        ,   0.        ,   1.        ]])

dist = np.array([[-0.31429013,  0.09988483, -0.00360046, -0.00090223, -0.01470093]])

newcameramtx = np.array([[ 39.06088638,   0.        , 285.18778276],
       [  0.        ,  60.61566544, 115.67541321],
       [  0.        ,   0.        ,   1.        ]])


roi = (239, 63, 84, 100)
    

def xy_points(index, width):
    M = cv2.moments(index*1.)
    cY = int(M["m10"] / M["m00"])
    cX = int(M["m01"] / M["m00"])
    if cX < 10:
        cX = 10
    #if cY < 10:
    #    cY = 10
    angle = np.arctan2(width/2-cY, width-cX)#*pi#-cX)
    return angle, cY, cX


def segmentation(image):
    #tri = triangle()
    #tri = tri[:,:]==1
    image = cv2.resize(image, (width,height))
    x_tensor = preprocess(image)
    mask = model.predict(x_tensor).to(device).half()
    mask = (mask.squeeze().cpu().numpy().round())
    mask = np.array(mask, dtype = np.uint8)#mask = np.uint8(mask)
    mask = mask == 1
    #mask = mask*tri
    mask = mask[:,:,np.newaxis]
    mask1 = np.concatenate([mask,mask,mask], axis =2)
    #way Segmentation
    index = np.concatenate([mask*128,mask*64,mask*128], axis =2)
    #return image*index, mask
    return image*mask1, index*1


def un_dst(image, width, height):
    #Undistortion parameters.
    image = cv2.resize(image, (400,400), interpolation = cv2.INTER_AREA)
    dst = cv2.undistort(image, mtx, dist, None, newcameramtx)
    # crop the image
    x,y,w,h = roi
    image = dst[y:y+h, x:x+w]
    return image#cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)

def show_segmented_view(image,cY,cX):
    image = cv2.circle(image, (cY,cX), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (int(width / 2), widget_height), 8, (0, 0,255), 3)
    image = cv2.line(image, (cY,cX), (int(width / 2), widget_height), (255,0,0), 3)
    #image = blend(image, colorized)
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [5]:
camera = Camera()

widget_width = 320#camera.width
widget_height = 320#camera.height

width, height = 160,160

image_widget = widgets.Image(format='jpeg',  width=widget_width, height=widget_height)
target_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.0, description='speed gain')
steering_Pgain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.17, description='steering kP')
steering_Dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.01, value=0.30, description='steering kD')
steering_Igain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.01, value=0.0, description='steering kI')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.005, value=0.0, description='steering bias')


x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
left_speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='left speed')
right_speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='right speed')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
direction_point = widgets.widget_float.Tuple


robot = Robot()

In [6]:


angle = 0.0
angle_last = 0.0


def display_xy(camera_image):
    global angle, angle_last
    image1 = np.copy(camera_image)
    image1 = np.resize(image1,(400,400))
    
    ## Need to be checked for the channel
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    ############## undistort
    image1 = un_dst(image1, width,height)
    ###############
    image, index1 = segmentation(image1)
    #####################
    angle, cY, cX = xy_points(np.squeeze(index1[:,:,0]),width)
    
    x_slider.value = cX/width/2
    y_slider.value = cY/height/2

    image = show_segmented_view(image,cY,cX)#show_segmented_view(image, direction_point.ValueWidget) 
    
    pid = angle * steering_Pgain_slider.value +  (angle - angle_last)*steering_Dgain_slider.value #+((angle + angle_last)/2)*steering_Igain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value

    robot.left_motor.value = max(min(speed_gain_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_gain_slider.value - steering_slider.value, 1.0), 0.0)    

    image = cv2.resize(image, (160,160), interpolation = cv2.INTER_AREA)

    
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

#time.sleep(3)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([target_widget]))

display(ipywidgets.HBox([y_slider,speed_slider, left_speed_slider, right_speed_slider]))
display(x_slider, steering_slider)


display(speed_gain_slider, steering_Pgain_slider, steering_Igain_slider, steering_Dgain_slider, steering_bias_slider)

#direction_point = widgets.widget_float.Tuple



FloatSlider(value=0.371875, description='x', max=1.0, min=-1.0)

FloatSlider(value=-0.011461142311177713, description='steering', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='speed gain', max=2.0, step=0.01)

FloatSlider(value=0.17, description='steering kP', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kI', max=0.5, step=0.01)

FloatSlider(value=0.3, description='steering kD', max=0.5, step=0.01)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.005)

In [7]:
robot.stop()


In [ ]:
camera.stop()